# Project 2
Lab journal

Aigul Nugmanova

Bogdan Sotnikov

## Day 1 02.11.22

### Step 1. The preparations.


We found the data about [this year vaccine strains of the flu](https://www.fda.gov/vaccines-blood-biologics/lot-release/influenza-vaccine-2022-2023-season). It contains 4 strains:

>    A/Victoria/2570/2019 (H1N1)pdm09-like virus;

>    A/Darwin/9/2021 (H3N2)-like virus;

>    B/Austria/1359417/2021-like virus (B/Victoria lineage); 

>    B/Phuket/3073/2013-like virus (B/Yamagata lineage).

The HI reactions result showed another substrain of flu: 
> A/Hong Kong/4801/2014 (H3N2)
### Step 2. Inspecting the raw data.

We downloaded data from [NCBI CRA](http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/001/SRR1705851/). We use bash command
```bash
wget http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/001/SRR1705851/
```

Then we downaloaded reference sequence for hemagglutinine from [NCBI](https://www.ncbi.nlm.nih.gov/nuccore/KF848938.1?report=fasta).



## Day 2 03.11.22
### Step 3. Analyzing data.

We used fastqc (v. 0.11.9) for analyzing quality of reads.
```bash
fastqc SRR1705851.fastq.gz
```

We had warnings in results of fastq analyssis in sections "Per base sequence content" and "Sequence Duplication Levels". Also we had some bourderline values of tests in "Per sequence GC content", "Sequence Length Distribution" and "Overrepresented sequences". 

Per sequence GC content bias in the start of figure may be caused by Illuminas' artifacts. The mismatching of nucleotide numbers may be caused of little genome amount.

Very high sequence duplication levels may be caused by problems in process of genome library amplification or by sequencing of little fragment of virus NA.

For clearing raw data we use trimmomatic (v. 0.39).
```bash
java -jar /usr/share/java/trimmomatic-0.39.jar SE -phred33 ./SRR1705851.fastq.gz ./SRR1705851_trimmed.fastq.gz LEADING:10 TRAILING:10 SLIDINGWINDOW:10:10
```
We hadn't seen any significant changes in the fastqc output and number of reads in our data, and next time we worked with first variant of fastq.

### Step 4. Alignment.

In the next step we indexed reference, aligned reads to reference and sorted alignment. For this purposes we used bwa (v. 0.7.17-r1188), samtools (v. 1.16.1) and snakemake (v. 7.17.1). We created file named Snakefile in the working directory and filled it:

```bash
rule bwa_index:
        input:
                "{reference}.fasta"
        output:
                "{reference}.fasta.amb",
                "{reference}.fasta.ann",
                "{reference}.fasta.bwt",
                "{reference}.fasta.pac",
                "{reference}.fasta.sa"
        shell:
                "bwa index {input}"

rule bwa_mem:
        input:
                "{reference}.fasta.amb",
                "{reference}.fasta.ann",
                "{reference}.fasta.bwt",
                "{reference}.fasta.pac",
                "{reference}.fasta.sa",
                ref = "{reference}.fasta",
                reads = "{sample}.fastq.gz"
        output:
                "{reference}.{sample}.unsorted.bam"
        shell:
                "bwa mem {input.ref} {input.reads} | samtools view -b > {output}"

rule sorting_bam:
        input:
                "{reference}.{sample}.unsorted.bam"
        output:
                "{reference}.{sample}.sorted.bam"
        shell:
                "samtools sort {input} -o {output}"

```

After exiting the vim (sorry for chestnut) we explored the script:
```bash
snakemake -p --cores all sequence.SRR1705851.sorted.bam
```

The next stage of our work was indexing the sorted file. The command was:
```bash
samtools index sequence.SRR1705851.sorted.bam
```

## Day 3. 07.11.22
### Step 5. Preparation for variant calling

For pilig up our data from sequncing artifacts we used samtools mpileup. For adequate using of it we need to calculate mean fof the coverage depth. We need to know the total number of mapped reads, average reads' length and length of the reference.

The total number of mapped reads we can know using samtools. The command and output are wroten below.

```bash
samtools flagstat sequence.SRR1705851.unsorted.bam


#361349 + 0 in total (QC-passed reads + QC-failed reads)
#358265 + 0 primary
#0 + 0 secondary
#3084 + 0 supplementary
#0 + 0 duplicates
#0 + 0 primary duplicates
#361116 + 0 mapped (99.94% : N/A)
#358032 + 0 primary mapped (99.93% : N/A)
#0 + 0 paired in sequencing
#0 + 0 read1
#0 + 0 read2
#0 + 0 properly paired (N/A : N/A)
#0 + 0 with itself and mate mapped
#0 + 0 singletons (N/A : N/A)
#0 + 0 with mate mapped to a different chr
#0 + 0 with mate mapped to a different chr (mapQ>=5)
```
Now we have data about number of mapped reads - 358032.

For data about average length of read we used python script. As first stage of analisis we gunzipped fastq file and then used belowlisted script.

```python
with open("<path_to_file>/SRR1705851.fastq", "r") as file_input:
    summator=[]

# We calculate len of every second string. It isn't a problem, beacuse legths of second and fourth strings are equal.

    for line in file_input:
        string = file_input.readline().strip()
        summator.append(len(string))
    mean = sum(summator)/len(summator)
    print(mean)

    #147.14768118571448
```

We calculate the reference length by another script.

```python    
with open("<path_to_file>/sequence.fasta", "r") as file_input:
    summator = []
    string = file_input.readline().strip()
    print(len(string))
!wc -m /sequence.fasta        

#103
#1794 <path_to_file>/sequence.fasta
```

Length of reference will be equal 1794 - 103 = 1691. Hence number we ssek is equal 
> (358032*147.15)/1691 = 31155.77

31156 is a threshold we need to give to samtools mpileup.


```bash
samtools mpileup -d 32000 -f sequence.fasta sequence.SRR1705851.sorted.bam > my.mpileup
```


### Step 6. Variant calling.

#### Substep 6.1. 95%

For variant calling we used VarScan (v. 2.3.9).
At the first stage we found most common SNPs (5) with 0.95 threshold.

```bash
java -jar <path_to_varscan>/VarScan.v2.3.9.jar  mpileup2snp my.mpileup --min-var-freq 0.95 --variants --output-vcf 1 > VarScan95_results.vcf



#KF848938.1	72	.	A	G	.	PASS	ADP=16794;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:16832:16794:6:16787:99.96%:0E0:35:36:4:2:10898:5889
#KF848938.1	117	.	C	T	.	PASS	ADP=20254;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:20352:20254:35:20217:99.82%:0E0:35:37:27:8:13382:6835
#KF848938.1	774	.	T	C	.	PASS	ADP=30514;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:30652:30514:5:30505:99.97%:0E0:34:37:5:0:17834:12671
#KF848938.1	999	.	C	T	.	PASS	ADP=28797;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:29226:28797:36:28756:99.86%:0E0:35:35:21:15:15647:13109
#KF848938.1	1260	.	A	C	.	PASS	ADP=23033;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:23067:23033:2:23019:99.94%:0E0:32:37:0:2:9824:13195
```
Or in more convinient format with awk:

```bash
cat VarScan95_results.vcf | awk 'NR>24 {print $1, $2, $4, $5}'


#KF848938.1 72 A G
#KF848938.1 117 C T
#KF848938.1 774 T C
#KF848938.1 999 C T
#KF848938.1 1260 A C
```

For analysing mutations we used IGV.

The first mutation was substitution of A for G. It substitutes of ACA triplet to ACG. It's a samesense-mutation, because the result of both triplet translation will be threonine.

The second mutation was substitution of C for T. It substitutes of GCC triplet to GCT. It's a samesense-mutation, because the result of both triplet translation will be alanine.

The third mutation was substitution of C for T. It substitutes of TTC triplet to TTT. It's a samesense-mutation, because the result of both triplet translation will be phenylalanine.

The fourth mutation was substitution of C for T. It substitutes of GGC triplet to GGT. It's a samesense-mutation, because the result of both triplet translation will be glycine.

The fifth mutation was substitution of A for C. It substitutes of CTA triplet to CTC. It's a samesense-mutation, because the result of both triplet translation will be leucine.

As a result of first stage we have only five samesense mutations.


#### Substep 6.2. 0.1%

We tried to find rare variants of SNPs.

```bash
java -jar ../../VarScan.v2.3.9.jar  mpileup2snp my.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > VarScan01_results.vcf

#KF848938.1	72	.	A	G	.	PASS	ADP=16794;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:16832:16794:6:16787:99.96%:0E0:35:36:4:2:10898:5889
#KF848938.1	117	.	C	T	.	PASS	ADP=20254;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:20352:20254:35:20217:99.82%:0E0:35:37:27:8:13382:6835
#KF848938.1	254	.	A	G	.	PASS	ADP=30830;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:27:30963:30830:30767:58:0.19%:1.8611E-3:36:36:21003:9764:36:22
#KF848938.1	307	.	C	T	.	PASS	ADP=28501;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:255:28604:28501:28224:272:0.95%:6.6218E-52:36:35:17685:10539:144:128
#KF848938.1	340	.	T	C	.	PASS	ADP=29279;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:21:29423:29279:29223:52:0.18%:6.9669E-3:37:35:18790:10433:34:18
#KF848938.1	389	.	T	C	.	PASS	ADP=26881;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:39:27041:26881:26817:61:0.23%:1.1057E-4:37:36:14104:12713:41:20
#KF848938.1	722	.	A	G	.	PASS	ADP=30017;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:41:30041:30017:29944:68:0.23%:7.7136E-5:37:36:19654:10290:39:29
#KF848938.1	744	.	A	G	.	PASS	ADP=30435;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:23:30477:30435:30371:55:0.18%:4.3947E-3:37:32:19521:10850:33:22
#KF848938.1	774	.	T	C	.	PASS	ADP=30514;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:30652:30514:5:30505:99.97%:0E0:34:37:5:0:17834:12671
#KF848938.1	802	.	A	G	.	PASS	ADP=31594;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:52:31685:31594:31513:77:0.24%:5.4834E-6:37:35:18099:13414:28:49
#KF848938.1	915	.	T	C	.	PASS	ADP=30957;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:32:31055:30957:30887:62:0.2%:5.508E-4:35:35:16615:14272:35:27
#KF848938.1	999	.	C	T	.	PASS	ADP=28797;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:29226:28797:36:28756:99.86%:0E0:35:35:21:15:15647:13109
#KF848938.1	1043	.	A	G	.	PASS	ADP=28440;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:26:28483:28440:28382:55:0.19%:2.0064E-3:35:33:15882:12500:18:37
#KF848938.1	1086	.	A	G	.	PASS	ADP=23991;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:31:23992:23991:23938:51:0.21%:7.5186E-4:36:35:12540:11398:21:30
#KF848938.1	1213	.	A	G	.	PASS	ADP=25093;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:34:25177:25093:25035:56:0.22%:3.7244E-4:37:36:8964:16071:24:32
#KF848938.1	1260	.	A	C	.	PASS	ADP=23033;WT=0;HET=0;HOM=1;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	1/1:255:23067:23033:2:23019:99.94%:0E0:32:37:0:2:9824:13195
#KF848938.1	1280	.	T	C	.	PASS	ADP=23462;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:20:23487:23462:23418:43:0.18%:9.2875E-3:37:35:11147:12271:24:19
#KF848938.1	1458	.	T	C	.	PASS	ADP=25707;WT=0;HET=1;HOM=0;NC=0	GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR	0/1:255:25838:25707:25490:214:0.83%:4.5784E-39:37:35:6834:18656:80:134
```



### Step 7. Inspecting and aligning the control sample sequencing data

We downloaded fastq data for three controls.

```bash
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/008/SRR1705858/SRR1705858.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/009/SRR1705859/SRR1705859.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/000/SRR1705860/SRR1705860.fastq.gz 
```

We calculated average genome coverage. The number of covered reads was 256500 in the first sample, 233251 in the second sample, 249888 in the third sample. Mean leangths of reads in fastq (calculated by above-described python script) are 148.56 for the first one, 148.45 for the second and 148.70 for the third.
The average coverages are 22534.4, 20476.7 and 21974.2 respectively.




Then we aligned all three sequences on the reference, using described above snakemake script.

```bash
snakemake -p --cores all sequence.SRR1705858.sorted.bam
snakemake -p --cores all sequence.SRR1705859.sorted.bam
snakemake -p --cores all sequence.SRR1705860.sorted.bam
```

All alignments were indexed by smatools.

```bash
samtools index sequence.SRR1705858.sorted.bam
samtools index sequence.SRR1705859.sorted.bam
samtools index sequence.SRR1705860.sorted.bam
```

And then we used mpileup in a above-described manner. 23000 was chosen as a threshold, because it's quite higher then genome coverage of our three alignments.

```bash
samtools mpileup -d 23000 -f sequence.fasta sequence.SRR1705858.sorted.bam > my58.mpileup
samtools mpileup -d 23000 -f sequence.fasta sequence.SRR1705859.sorted.bam > my59.mpileup
samtools mpileup -d 23000 -f sequence.fasta sequence.SRR1705860.sorted.bam > my60.mpileup
```

At the next stage all alignments were elaborated with varscan.

```bash
java -jar ../../VarScan.v2.3.9.jar  mpileup2snp my58.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > VarScan5801_results.vcf

java -jar ../../VarScan.v2.3.9.jar  mpileup2snp my59.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > VarScan5901_results.vcf

java -jar ../../VarScan.v2.3.9.jar  mpileup2snp my60.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > VarScan6001_results.vcf
```

For parsing our samples we created two TSV files for each vcf file: first contained data about reference, locus of substitution, and two nucleotids: before and after substitution, and second contained probability of this substitution. Then we combined them and computed two types of threshold - 1-tailed t-distribution with p=0.05 and 3 SDs from the each mean.

```bash
cat VarScan5801_results.vcf | awk -v FS="\t" -v OFS="\t" 'NR>24 {print $1, $2, $4, $5}' > positions58.tsv
cat VarScan5901_results.vcf | awk -v FS="\t" -v OFS="\t" 'NR>24 {print $1, $2, $4, $5}' > positions59.tsv
cat VarScan6001_results.vcf | awk -v FS="\t" -v OFS="\t" 'NR>24 {print $1, $2, $4, $5}' > positions60.tsv

cat VarScan5801_results.vcf | awk 'NR>24 {print $10}' | awk -v FS=":" -v OFS="\t" '{print $7}' > possibility58.tsv
cat VarScan5901_results.vcf | awk 'NR>24 {print $10}' | awk -v FS=":" -v OFS="\t" '{print $7}' > possibility59.tsv
cat VarScan6001_results.vcf | awk 'NR>24 {print $10}' | awk -v FS=":" -v OFS="\t" '{print $7}' > possibility60.tsv
```

On the next stage we computed three means and three standard deviations of mutation probability.

```R
# Setting working directory
setwd('<path_to_directory>')

# Reading data
nucl58 <- read.csv("positions58.tsv", sep = "\t", header = FALSE)
nucl59 <- read.csv("positions59.tsv", sep = "\t", header = FALSE)
nucl60 <- read.csv("positions60.tsv", sep = "\t", header = FALSE)
prob58 <- read.csv("possibility58.tsv", sep = "\t", header = FALSE)
prob59 <- read.csv("possibility59.tsv", sep = "\t", header = FALSE)
prob60 <- read.csv("possibility60.tsv", sep = "\t", header = FALSE)

# Removing "%" at the end of strings
prob58 <- apply(prob58, 1, function(x) gsub("%", "", x))
prob59 <- apply(prob59, 1, function(x) gsub("%", "", x))
prob60 <- apply(prob60, 1, function(x) gsub("%", "", x))

# Merging tables
result58 <- cbind(nucl58, prob58)
result59 <- cbind(nucl59, prob59)
result60 <- cbind(nucl60, prob60)

# Transforming probability from strings to numerical data
result58$prob58 <- as.double(result58$prob58)
result59$prob59 <- as.double(result59$prob59)
result60$prob60 <- as.double(result60$prob60)


# Computing statistics and finding threshold for mutation probability
mean(result58$prob58)
sd(result58$prob58)

critical58_1 = mean(result58$prob58)+(1.6741*(sd(result58$prob58)/(length(result58$prob58)**0.5)))
critical58_1
critical58_2 = mean(result58$prob58)+(sd(result58$prob58)*3)
critical58_2

mean(result59$prob59)
sd(result59$prob59)

critical59 = mean(result59$prob59)+(1.6772*(sd(result59$prob59)/(length(result59$prob59)**0.5)))
critical59
critical59_2 = mean(result59$prob59)+(sd(result59$prob59)*3)
critical59_2


mean(result60$prob60)
sd(result60$prob60)

critical60 = mean(result60$prob60)+(1.6725*(sd(result60$prob60)/(length(result60$prob60)**0.5)))
critical60
critical60_2 = mean(result60$prob60)+(sd(result60$prob60)*3)
critical60_2



# 0.2782936
# 0.4761118
# 0.2543942
# 0.3967154
# 0.2730022
# 0.4932908
```
As a slightly paranoidal person, I use maximum value as a threshold for VarScan.

We created new VCF file from experimental data with threshold 0.493. 

```bash
java -jar ../../VarScan.v2.3.9.jar  mpileup2snp my.mpileup --min-var-freq 0.00493 --variants --output-vcf 1 > VarScan04_results.vcf
```
As a result, we have had 7 SNPs. 5 of them are high-frequency variants, that were analised before. For analysing two another SNPs we used IGV.

The first mutation was substitution of C for T. It substitutes of CCG triplet to TCG. It's a missense-mutation, because the result of translation is serine instead of proline. This SNP is situated on the 307th nucleotide position, 103th aminoacid, epitope D of hemagglutinine.

```
KF848938.1:1-1653

Type: gene
ID: gene-HA
Name: HA
gbkey: Gene
gene: HA
gene_biotype: protein_coding
partial: true
start_range: .,1
---------------------------
Type: CDS
ID: cds-AHB59323.1
Parent: gene-HA
Dbxref: NCBI_GP:AHB59323.1
Name: AHB59323.1
gbkey: CDS
gene: HA
partial: true
product: hemagglutinin
protein_id: AHB59323.1
start_range: .,1
```

The second mutation was substitution of T for C. It substitutes of TAT triplet to TAC. It's a samesense-mutation, because the result of both triplet translation will be tyrosine.

